#01.Import Libraries

In [3]:
#import libraries
import pandas as pd
import numpy as np
import os
import seaborn as sns
import scipy

#02.Import Data

In [4]:
#create path
path = r'/Users/juliewhitton/Desktop/Instacart Basket Analysis/'

In [5]:
#import customers.csv
df_customers = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'customers.csv'))

In [6]:
#check import
df_customers.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


#03.Wrangle new df

In [7]:
#check for missing values
df_customers.apply(lambda x: x.isnull().value_counts())

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
False,206209.0,194950,206209.0,206209.0,206209.0,206209.0,206209.0,206209.0,206209.0,206209.0
True,NaN,11259,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
#change column names to be more uniform 
df_customers.rename(columns = {'First Name' : 'first_name', 'Surnam' : 'last_name', 'Gender' : 'gender', 'STATE' : 'state','Age' : 'age',
                               'n_dependants' : 'number_dependents', 'fam_status' : 'family_status'}, 
                    inplace = True)

#check col name changes
df_customers.head()


,user_id,first_name,last_name,gender,state,age,date_joined,number_dependents,family_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [9]:
#change data type of user_id and first_name
df_customers[['user_id', 'first_name']] = df_customers[['user_id', 'first_name']].astype('str')

#04.Data Quality and Consistency Checks

In [10]:
#check for mixed type data
for col in df_customers.columns.tolist():
  weird = (df_customers[[col]].applymap(type) != df_customers[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_customers[weird]) > 0:
    print (col)

#05.Combine data sets

In [12]:
#import existing ords_prods
ords_prods = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 
                                         'ords_prods_viz.pkl'))

#change data type of user_id in ords_prods to match df_customer
ords_prods['user_id'] = ords_prods['user_id'].astype('str')

#check import with head and shape.head()
ords_prods.head()

,order_id,user_id,eval_set,order_number,order_day_of_week,time_of_order,days_since_prior_order,product_id,add_to_cart_order,reordered,...,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg_spent,spending_flag,median_days_since_last_order,freq_flag
0,2539329,1,prior,1,2,8,NaN,196,1,0,...,Mid-range product,Regularly busy,Regularly Busy Days,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
1,2398795,1,prior,2,3,7,15.0,196,1,1,...,Mid-range product,Regularly busy,Least Busy Days,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
2,473747,1,prior,3,3,12,21.0,196,1,1,...,Mid-range product,Regularly busy,Least Busy Days,Most orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
3,2254736,1,prior,4,4,7,29.0,196,1,1,...,Mid-range product,Least busy day,Least Busy Days,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
4,431534,1,prior,5,4,15,28.0,196,1,1,...,Mid-range product,Least busy day,Least Busy Days,Most orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer


In [13]:
ords_prods.shape

(32404859, 24)

In [14]:
#check shape of customers
df_customers.shape

(206209, 10)

In [15]:
#merge dfs
full_data_set = ords_prods.merge(df_customers, on = ['user_id'], indicator = True).drop(columns = ['_merge'])
full_data_set.head()

,order_id,user_id,eval_set,order_number,order_day_of_week,time_of_order,days_since_prior_order,product_id,add_to_cart_order,reordered,...,freq_flag,first_name,last_name,gender,state,age,date_joined,number_dependents,family_status,income
0,2539329,1,prior,1,2,8,NaN,196,1,0,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
1,2398795,1,prior,2,3,7,15.0,196,1,1,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
2,473747,1,prior,3,3,12,21.0,196,1,1,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
3,2254736,1,prior,4,4,7,29.0,196,1,1,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
4,431534,1,prior,5,4,15,28.0,196,1,1,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423


In [16]:
full_data_set.apply(lambda x: x.isnull().value_counts())

,order_id,user_id,eval_set,order_number,order_day_of_week,time_of_order,days_since_prior_order,product_id,add_to_cart_order,reordered,...,freq_flag,first_name,last_name,gender,state,age,date_joined,number_dependents,family_status,income
False,32404859.0,32404859.0,32404859.0,32404859.0,32404859.0,32404859.0,30328763,32404859.0,32404859.0,32404859.0,...,32404854,32404859.0,32404859.0,32404859.0,32404859.0,32404859.0,32404859.0,32404859.0,32404859.0,32404859.0
True,NaN,NaN,NaN,NaN,NaN,NaN,2076096,NaN,NaN,NaN,...,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
full_data_set.shape

(32404859, 33)

In [18]:
#export full_data_set
full_data_set.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'full_data_set.pkl'))